# Preparación modulos propios

In [1]:
import config
import lab
import pre_process
import reduction
import metrics
import audio
import numpy as np
import visualization
import ui
from pre_process import get_chord_type_from_intervals, clean_vector, safe_normalize, Acorde, ChordAdapter, ModeloRugosidad, ModeloSethares, ModeloEuler, ModeloArmonicosCriticos, ResultadoExperimento
from typing import Dict, Optional
from dataclasses import dataclass
import chordcodex



✅ lab.py: Clase 'LaboratorioAcordes' actualizada y lista para ejecutar experimentos.
✅ Módulo visualization.py cargado con la nueva lógica de color transformado.


## Modelos

In [2]:
# CELDA 6: Uso de las Implementaciones Concretas de Modelos de Rugosidad Musical importadas desde pre_process.py


# --- Configuraciones de Ejemplo para cada Modelo ---
# Para ModeloSethares, usamos algunos parámetros de ejemplo (se usarán los valores por defecto si no se especifica alguno)

# Para ModeloSethares: Usamos configuración vacía para que se usen los valores por defecto.
modelo_sethares = ModeloSethares({})

"""
# Para ModeloEuler, dejamos la configuración vacía para que use sus valores por defecto
config_euler = {}
modelo_euler = ModeloEuler(config_euler)

# Para ModeloArmonicosCriticos, definimos la frecuencia base de ejemplo
config_armonicos = {'base_freq': 440.0}
modelo_armonicos = ModeloArmonicosCriticos(config_armonicos)
"""
# --- Creación de un Acorde de Ejemplo ---
# Basado en tus datos reales, tomamos un acorde con intervalos [4,3] y algunas frecuencias.
acorde_test = Acorde(
    name="Ejemplo",
    intervals=[7],
    chroma=np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    frequencies=[261.63,  261.63*2]  # Ejemplo: C (261.63 Hz), E (329.63 Hz), G (392.00 Hz)
)
frecbase=261.63


for i in range(13):
    frec2=frecbase * (2**((i)/12))
    test=modelo_sethares._calcular_disonancia_pairwise(frecbase, frec2,1,1)
    print(frecbase, frec2,test)





# --- Evaluación de cada Modelo ---
print("Ejemplo - Modelo Sethares:")
vector_sethares, rough_sethares = modelo_sethares.calcular(acorde_test)
print("Vector de rugosidad:", vector_sethares)
print("Rugosidad total:", rough_sethares)
"""
print("\nEjemplo - Modelo Euler:")
vector_euler, rough_euler = modelo_euler.calcular(acorde_test)
print("Vector de rugosidad:", vector_euler)
print("Rugosidad total:", rough_euler)

print("\nEjemplo - Modelo Armónicos Críticos:")
vector_armonicos, rough_armonicos = modelo_armonicos.calcular(acorde_test)
print("Vector de rugosidad:", vector_armonicos)
print("Rugosidad total:", rough_armonicos)

print("✅ CELDA 6: Modelos de rugosidad importados y probados correctamente.")
"""


261.63 261.63 0.0
261.63 277.18732937722245 0.8482669819498088
261.63 293.66974569918125 0.8372201090564029
261.63 311.1322574981619 0.6003646963857276
261.63 329.63314428399565 0.3703813533742357
261.63 349.2341510465061 0.20710214294673304
261.63 370.00069432367286 0.10733140004095176
261.63 392.0020805232462 0.05212371816906928
261.63 415.31173722644 0.023850452639247954
261.63 440.00745824565865 0.01030776734739798
261.63 466.1716632541139 0.00420974098801891
261.63 493.89167285382297 0.0016233544214949755
261.63 523.26 0.0005900259092319747
Ejemplo - Modelo Sethares:
Vector de rugosidad: [0.         0.         0.         0.         0.         0.
 0.03947149 0.         0.         0.         0.         0.        ]
Rugosidad total: 0.03947148929715852


'\nprint("\nEjemplo - Modelo Euler:")\nvector_euler, rough_euler = modelo_euler.calcular(acorde_test)\nprint("Vector de rugosidad:", vector_euler)\nprint("Rugosidad total:", rough_euler)\n\nprint("\nEjemplo - Modelo Armónicos Críticos:")\nvector_armonicos, rough_armonicos = modelo_armonicos.calcular(acorde_test)\nprint("Vector de rugosidad:", vector_armonicos)\nprint("Rugosidad total:", rough_armonicos)\n\nprint("✅ CELDA 6: Modelos de rugosidad importados y probados correctamente.")\n'

## Cargar funciones de los modulo


In [3]:
from audio import(
    ChordPlayer,

)

from lab import(
    LaboratorioAcordes,
    ResultadoExperimento,
)
from metrics import(
    compute_trustworthiness,
    compute_knn_recall,
    compute_continuity,
    reporte_parametros_reduccion,

)

from reduction import(
    kernel_mds,
    laplacian_eigenmaps_embedding,
    geometric_mds_vectorized,
    calculate_stress_vectorized,
    laplacian_eigenmaps_embedding,

)

# Cargar DB

In [11]:
# %% [code]
"""
CELDA 11: Carga de Datos de Acordes desde la Base de Datos con ChordCodex (Refactorizada)

Descripción:
Esta celda se encarga de la carga de datos de acordes desde la base de datos PostgreSQL
utilizando la biblioteca `chordcodex`. Se instancia `QueryExecutor` con las credenciales
de conexión definidas en Celda 1 y se ejecuta una consulta SQL (también definida en Celda 1)
para obtener los datos de acordes de 3 notas. Los resultados se cargan directamente
en un DataFrame de pandas para su posterior procesamiento.

Refactorización Clave:
    - Uso exclusivo de `chordcodex.QueryExecutor` para la gestión de la conexión
      y la ejecución de consultas a la base de datos, eliminando la necesidad
      de código manual de conexión con `psycopg2`.
    - Centralización de la consulta SQL en la constante global `QUERY_CHORDS_3_NOTES`
      definida en Celda 1, mejorando la organización y facilitando la modificación
      de la consulta en un solo lugar.
    - Simplificación del código de carga de datos, haciéndolo más conciso y legible
      mediante el uso de las herramientas de `chordcodex`.

Funcionalidades:
    - Inicialización de `QueryExecutor` con la configuración de la base de datos
      definida en Celda 1.
    - Ejecución de la consulta SQL `QUERY_CHORDS_3_NOTES` (definida en Celda 1)
      utilizando `QueryExecutor.as_pandas()`.
    - Carga de los resultados directamente en un DataFrame de pandas llamado `df_filtered_chords`.
    - Verificación de la carga de datos mediante la impresión del número de registros
      y la visualización de los primeros registros y tipos de datos del DataFrame.
      config_db2 = {
    "host": DB_HOST,
    "port": int(DB_PORT),  # Convertir a entero
    "user": DB_USER,
    "password": DB_PASSWORD,
    "dbname": DB_NAME
}
"""

import pandas as pd
from chordcodex.model import QueryExecutor  # Importar QueryExecutor de chordcodex
from config import config_db, QUERY_CHORDS_3_NOTES, QUERY_TRIADS_WITH_INVERSIONS, QUERY_CHORDS_WITH_NAME, QUERY_CHORDS_WITH_NAME_AND_RANDOM_CHORDS_POBLATION, QUERY_CHORDS_SPECIFIC_INTERVALS_AND_RANDOM_SAME_OCTAVE

# --- Inicialización de QueryExecutor ---


query_executor = QueryExecutor(**config_db) # Inicializar QueryExecutor con la configuración

print("✅ QueryExecutor inicializado.")

# --- Ejecución de la consulta SQL y carga en DataFrame ---
# Consulta SQL para obtener acordes de 3 notas (definida como constante global en Celda 1)
query_acordes_3_notas = QUERY_CHORDS_3_NOTES
query_triads_with_inversions = QUERY_TRIADS_WITH_INVERSIONS
query_chords_with_name = QUERY_CHORDS_WITH_NAME
QUERY_TRIADS_ROOT_ONLY_MOBIUS_MAZZOLA = """
SELECT id, n, interval, notes, bass, octave, frequencies, chroma, tag, code
FROM chords
WHERE n = 3
  AND interval IN (
      ARRAY[4,3],  -- triada mayor
      ARRAY[3,4],  -- triada menor
      ARRAY[3,3]   -- triada disminuida
  )
  AND notes <@ ARRAY['0','2','4','5','7','9','B']::varchar[]
ORDER BY notes, id;
"""

QUERY_RANDOM_CHORD_MORE_THAN_2_NOTES = """
SELECT id, n, interval, notes, bass, octave, frequencies, chroma, tag, code
FROM chords
WHERE n > 2
ORDER BY RANDOM()
LIMIT 100;
"""
QUERY_RANDOM_CHORD_MORE_THAN_2_NOTES_CLOSE_OCTAVE = """
SELECT id, n, interval, notes, bass, octave, frequencies, chroma, tag, code
FROM chords
WHERE n >= 2
     AND ('x' || notes[1])::bit(32)::int = 0
     AND (SELECT SUM(i) FROM unnest(interval) AS i) <= 12
ORDER BY RANDOM()
LIMIT 900;
"""

repetidas = """
SELECT id, n, interval, notes, bass, octave, frequencies, chroma, tag, code
FROM chords
WHERE n = 3
  AND ('x' || notes[1])::bit(32)::int = 0
  AND (SELECT SUM(i) FROM unnest(interval) AS i) <= 12
  AND (
    SELECT COUNT(*) FROM (
      SELECT unnest(notes) AS note
    ) sub
    GROUP BY note
    HAVING COUNT(*) > 1
    LIMIT 1
  ) IS NOT NULL
ORDER BY RANDOM()
LIMIT 100;

"""
AcordesExtremos = """
(
  -- 🔥 Acorde con 10 notas y todos los intervalos = 1 (clúster extremo)
  SELECT id, n, interval, notes, bass, octave, frequencies, chroma, tag, code
  FROM chords
  WHERE n = 10
    AND interval = ARRAY[1,1,1,1,1,1,1,1,1]
)
UNION
(
  -- 🎯 Otros acordes con 10 notas y solo intervalos de 1 o 2 semitonos
  SELECT id, n, interval, notes, bass, octave, frequencies, chroma, tag, code
  FROM chords
  WHERE n = 10
    AND (
      SELECT bool_and(i BETWEEN 1 AND 2)
      FROM unnest(interval) AS i
    )
    AND interval != ARRAY[1,1,1,1,1,1,1,1,1]  -- evitar duplicar el clúster extremo
  ORDER BY RANDOM()
  LIMIT 30
);
"""
diadas="""
WITH octava AS (
  SELECT id, n, interval, notes, bass, octave, frequencies, chroma, tag, code
  FROM chords
  WHERE n = 2
    AND notes = ARRAY['0', '0']
    AND ('x' || notes[1])::bit(32)::int = 0
    LIMIT 1
),
all_dyads AS (
  SELECT id, n, interval, notes, bass, octave, frequencies, chroma, tag, code,
         interval[1] AS total_interval
  FROM chords
  WHERE n = 2
    AND notes != ARRAY['0', '0']
    AND ('x' || notes[1])::bit(32)::int = 0
    AND (SELECT SUM(i) FROM unnest(interval) AS i) BETWEEN 1 AND 11
),
ranked_dyads AS (
  SELECT id, n, interval, notes, bass, octave, frequencies, chroma, tag, code, total_interval,
         ROW_NUMBER() OVER (PARTITION BY total_interval ORDER BY RANDOM()) AS rn
  FROM all_dyads
)
SELECT id, n, interval, notes, bass, octave, frequencies, chroma, tag, code
FROM ranked_dyads
WHERE rn = 1
UNION ALL
SELECT * FROM octava
ORDER BY code;


"""

query_chords_with_name_and_random_chords_poblation = QUERY_CHORDS_WITH_NAME_AND_RANDOM_CHORDS_POBLATION
column_names = ["id", "n", "interval", "notes", "bass", "octave", "frequencies", "chroma", "tag", "code"]  # Nombres de columnas esperados


✅ QueryExecutor inicializado.


In [12]:
# Catálogo de consultas base (diadas → triadas → séptimas)
# Mantiene una muestra estable: una fila por intervalo o calidad, sin ordenar al azar.
QUERY_DYADS_REFERENCE = """
WITH unison AS (
    SELECT
        c.id, c.n, c.interval, c.notes, c.bass, c.octave, c.frequencies,
        c.chroma, c.tag, c.code,
        'unison' AS quality,
        0 AS semitone,
        ROW_NUMBER() OVER (ORDER BY c.octave, c.id) AS rn
    FROM chords c
    WHERE c.n = 2
      AND c.notes = ARRAY['0','0']::varchar[]
),
non_unison AS (
    SELECT
        c.id, c.n, c.interval, c.notes, c.bass, c.octave, c.frequencies,
        c.chroma, c.tag, c.code,
        'dyad' AS quality,
        c.interval[1] AS semitone,
        ROW_NUMBER() OVER (
            PARTITION BY c.interval[1]
            ORDER BY c.octave, c.id
        ) AS rn
    FROM chords c
    WHERE c.n = 2
      AND c.notes[1] = '0'
      AND c.notes <> ARRAY['0','0']::varchar[]
      AND array_length(c.interval, 1) = 1
      AND c.interval[1] BETWEEN 1 AND 12
)
SELECT *
FROM (
    SELECT * FROM unison WHERE rn = 1
    UNION ALL
    SELECT * FROM non_unison WHERE rn = 1
) AS pick
ORDER BY semitone;
"""

QUERY_TRIADS_CORE = """
WITH triad_catalog(quality, intervals) AS (
    VALUES
        ('Maj', ARRAY[4,3]::integer[]),
        ('Min', ARRAY[3,4]::integer[]),
        ('Dim', ARRAY[3,3]::integer[]),
        ('Aug', ARRAY[4,4]::integer[]),
        ('Sus2', ARRAY[2,5]::integer[]),
        ('Sus4', ARRAY[5,2]::integer[])
),
ranked AS (
    SELECT
        c.id, c.n, c.interval, c.notes, c.bass, c.octave, c.frequencies,
        c.chroma, c.tag, c.code,
        triad_catalog.quality,
        c.notes[1] AS root,
        ROW_NUMBER() OVER (
            PARTITION BY triad_catalog.quality, c.notes[1]
            ORDER BY c.octave, c.id
        ) AS rn
    FROM chords c
    JOIN triad_catalog ON c.interval = triad_catalog.intervals
    WHERE c.n = 3
)
SELECT *
FROM ranked
WHERE rn = 1
ORDER BY quality, root;
"""

QUERY_SEVENTHS_CORE = """
WITH seventh_catalog(quality, intervals) AS (
    VALUES
        ('7', ARRAY[4,3,3]::integer[]),
        ('m7', ARRAY[3,4,3]::integer[]),
        ('Maj7', ARRAY[4,3,4]::integer[]),
        ('m7b5', ARRAY[3,3,4]::integer[]),
        ('Dim7', ARRAY[3,3,3]::integer[])
),
ranked AS (
    SELECT
        c.id, c.n, c.interval, c.notes, c.bass, c.octave, c.frequencies,
        c.chroma, c.tag, c.code,
        seventh_catalog.quality,
        c.notes[1] AS root,
        ROW_NUMBER() OVER (
            PARTITION BY seventh_catalog.quality, c.notes[1]
            ORDER BY c.octave, c.id
        ) AS rn
    FROM chords c
    JOIN seventh_catalog ON c.interval = seventh_catalog.intervals
    WHERE c.n = 4
)
SELECT *
FROM ranked
WHERE rn = 1
ORDER BY quality, root;
"""

sample_sets = {
    "df_dyads_reference": QUERY_DYADS_REFERENCE,
    "df_triads_canonical": QUERY_TRIADS_CORE,
    "df_sevenths_canonical": QUERY_SEVENTHS_CORE,
}

results = {}
for df_name, sql in sample_sets.items():
    try:
        results[df_name] = query_executor.as_pandas(sql)
    except RuntimeError as exc:
        print(f"⚠️ Error al ejecutar {df_name}: {exc}")
    else:
        globals()[df_name] = results[df_name]
        print(f"✅ {df_name}: {len(results[df_name])} filas")

from IPython.display import display
if "df_dyads_reference" in results:
    display(results["df_dyads_reference"][ ["semitone", "notes", "code"] ])
if "df_triads_canonical" in results:
    display(results["df_triads_canonical"][ ["quality", "notes", "code"] ])
if "df_sevenths_canonical" in results:
    display(results["df_sevenths_canonical"][ ["quality", "notes", "code"] ])


✅ df_dyads_reference: 12 filas
✅ df_triads_canonical: 6 filas
✅ df_sevenths_canonical: 5 filas


,semitone,notes,code
0,0,"[0, 0]",00
1,1,"[0, 1]",01
2,2,"[0, 2]",02
3,3,"[0, 3]",03
4,4,"[0, 4]",04
5,5,"[0, 5]",05
6,6,"[0, 6]",06
7,7,"[0, 7]",07
8,8,"[0, 8]",08
9,9,"[0, 9]",09


,quality,notes,code
0,Aug,"[0, 4, 8]",048
1,Dim,"[0, 3, 6]",036
2,Maj,"[0, 4, 7]",047
3,Min,"[0, 3, 7]",037
4,Sus2,"[0, 2, 7]",027
5,Sus4,"[0, 5, 7]",057


,quality,notes,code
0,7,"[0, 4, 7, 10]",047A
1,Dim7,"[0, 3, 6, 9]",0369
2,m7,"[0, 3, 7, 10]",037A
3,m7b5,"[0, 3, 6, 10]",036A
4,Maj7,"[0, 4, 7, 11]",047B


# Query


In [6]:
df_1= query_executor.as_pandas(QUERY_TRIADS_ROOT_ONLY_MOBIUS_MAZZOLA)
df_2= query_executor.as_pandas(QUERY_CHORDS_WITH_NAME)
df_3= query_executor.as_pandas(QUERY_RANDOM_CHORD_MORE_THAN_2_NOTES_CLOSE_OCTAVE)
df_4= query_executor.as_pandas(repetidas)
df_5= query_executor.as_pandas(query_triads_with_inversions)
df_6= query_executor.as_pandas(AcordesExtremos)
#df_7= query_executor.as_pandas(diadas)

In [18]:
df_filtered_chords = pd.concat([ df_dyads_reference, df_2, df_6,df_3],  ignore_index=True)


In [20]:


# --- Verificación de la carga de datos ---
num_records = len(df_filtered_chords)
print(f"📊 Número de acordes de 3 notas cargados desde la base de datos: {num_records}")

if num_records > 0:
    print("\nPrimeros registros de acordes cargados:")
    display(df_filtered_chords.head(100))
    print("\nTipos de datos de las columnas:")
    display(df_filtered_chords.dtypes)
else:
    print("⚠️ No se cargaron datos de acordes. Revise la conexión a la base de datos y la consulta SQL.")

print("✅ CELDA 11: Carga de datos de acordes desde la base de datos con ChordCodex completada.")

📊 Número de acordes de 3 notas cargados desde la base de datos: 973

Primeros registros de acordes cargados:


,id,n,interval,notes,bass,octave,frequencies,chroma,tag,code,quality,semitone,rn,span_semitones,abs_mask_int,abs_mask_hex,notes_abs_json
0,12,2,[12],"[0, 0]",0,4,"[261.63, 523.26]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",ABS_V2,00,unison,0.0,1.0,NaN,NaN,NaN,NaN
1,1,2,[1],"[0, 1]",0,4,"[261.63, 277.18]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",ABS_V2,01,dyad,1.0,1.0,NaN,NaN,NaN,NaN
2,2,2,[2],"[0, 2]",0,4,"[261.63, 293.66]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",ABS_V2,02,dyad,2.0,1.0,NaN,NaN,NaN,NaN
3,3,2,[3],"[0, 3]",0,4,"[261.63, 311.13]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",ABS_V2,03,dyad,3.0,1.0,NaN,NaN,NaN,NaN
4,4,2,[4],"[0, 4]",0,4,"[261.63, 329.63]","[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",ABS_V2,04,dyad,4.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,79045,7,"[1, 3, 1, 3, 1, 2]","[0, 1, 4, 5, 8, 9, 11]",0,4,"[261.63, 277.18, 329.63, 349.23, 415.3, 440.0,...","[1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1]",ABS_V2,014589B,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,99122,7,"[2, 1, 1, 1, 2, 1]","[0, 2, 3, 4, 5, 7, 8]",0,4,"[261.63, 293.66, 311.13, 329.63, 349.23, 391.9...","[1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0]",ABS_V2,0234578,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,4330,5,"[2, 2, 2, 6]","[0, 2, 4, 6, 0]",0,4,"[261.63, 293.66, 329.63, 369.99, 523.26]","[1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]",ABS_V2,02460,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,32471,6,"[2, 1, 5, 2, 1]","[0, 2, 3, 8, 10, 11]",0,4,"[261.63, 293.66, 311.13, 415.3, 466.16, 493.88]","[1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1]",ABS_V2,0238AB,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Tipos de datos de las columnas:


id                  int64
n                   int64
interval           object
notes              object
bass               object
octave              int64
frequencies        object
chroma             object
tag                object
code               object
quality            object
semitone          float64
rn                float64
span_semitones    float64
abs_mask_int      float64
abs_mask_hex       object
notes_abs_json     object
dtype: object

✅ CELDA 11: Carga de datos de acordes desde la base de datos con ChordCodex completada.


In [16]:
# %% [code]
"""
CELDA 12: Preparación de Datos para el Laboratorio de Acordes (Refactorizada)

Descripción:
Esta celda toma el DataFrame `df_filtered_chords`, cargado en la Celda 11,
y lo transforma en una lista de objetos `Acorde`. Esta lista de objetos `Acorde`
es luego utilizada para instanciar la clase `LaboratorioAcordes`, preparando
así los datos en el formato requerido para la ejecución de experimentos de
modelado de rugosidad.

Procesamiento de Datos:
    - Conversión de DataFrame a Lista de Acordes:
        Itera sobre cada fila del DataFrame `df_filtered_chords` y utiliza la clase
        `ChordAdapter` para convertir cada fila en un objeto de la clase `Acorde`.
        Los objetos `Acorde` encapsulan la información de cada acorde de manera
        estructurada (nombre, intervalos, vector chroma).
    - Creación del Laboratorio de Acordes:
        Instancia la clase `LaboratorioAcordes` pasando la lista de objetos `Acorde`
        creada en el paso anterior. La instancia de `LaboratorioAcordes` se almacena
        en la variable global `laboratorio_total`, que será utilizada en celdas
        posteriores para ejecutar y gestionar experimentos.

Variables Globales Creadas:
    - acordes_totales (List[Acorde]): Lista que contiene los objetos `Acorde`
      creados a partir de los datos del DataFrame.
    - laboratorio_total (LaboratorioAcordes): Instancia de la clase `LaboratorioAcordes`
      inicializada con la lista de acordes preparados. Esta instancia gestiona
      la colección de acordes para los experimentos.
"""

# --- Preparación de la lista para objetos Acorde ---
acordes_totales = []  # Inicializar una lista vacía para almacenar objetos Acorde

# Iterar sobre cada fila del DataFrame df_filtered_chords
for _, row in df_filtered_chords.iterrows():
    acorde = ChordAdapter.from_csv_row(row)  # Usar ChordAdapter para crear objeto Acorde desde la fila
    acordes_totales.append(acorde)          # Añadir objeto Acorde a la lista

print(f"✅ {len(acordes_totales)} objetos 'Acorde' creados a partir del DataFrame.")

# --- Instanciar LaboratorioAcordes ---
laboratorio_total = LaboratorioAcordes(acordes_totales) # Instanciar LaboratorioAcordes con la lista de acordes

print("✅ Instancia de 'LaboratorioAcordes' creada y lista para experimentos.")
print("   Variable 'laboratorio_total' contiene el laboratorio de acordes.")

print("✅ CELDA 12: Preparación de datos y creación del laboratorio de acordes completadas.")

✅ 73 objetos 'Acorde' creados a partir del DataFrame.
✅ Instancia de 'LaboratorioAcordes' creada y lista para experimentos.
   Variable 'laboratorio_total' contiene el laboratorio de acordes.
✅ CELDA 12: Preparación de datos y creación del laboratorio de acordes completadas.


# Visualizador de verdad

In [ ]:
from ui import UI


ui = UI(laboratorio_total)
ui.display_ui()